In [1]:
from src.clp_zne.qpu_info.layout_cycles import heron_layout_cycles_12q
from src.clp_zne.hamiltonians import sherrington_kirkpatrick_model
from src.clp_zne.qpu_info.backends import FakeTorino
from src.clp_zne.mitigate import CLP_ZNE_mitigate
from src.clp_zne.utils import compute_evals_ideal
from qiskit_aer.noise import NoiseModel
from qiskit.circuit.library import TwoLocal
from tqdm import tqdm
import numpy as np
import pickle

In [3]:
backend = FakeTorino()
n_qubits = 12
observables = [sherrington_kirkpatrick_model(n_qubits, h=1, seed=i) for i in range(100)]

circuits = []
for i in range(20):
    circ = TwoLocal(n_qubits, ['rx', 'rz'], 'cz', entanglement="circular", reps=3)
    rng = np.random.default_rng(i)
    parameters = rng.uniform(-np.pi, np.pi, circ.num_parameters)
    circ.assign_parameters(parameters, inplace=True)
    circuits.append(circ)

In [ ]:
noise_model = NoiseModel.from_backend(backend, readout_error=False, gate_error=True, thermal_relaxation=True)

layouts = [heron_layout_cycles_12q[i] for i in [5, 9, 13, 16, 7]]

all_evals_ideal = []
all_evals_mitigated = []
all_evals_noisy = []
all_error_sums = []

for circ in tqdm(circuits):
    evals_ideal = compute_evals_ideal(circ, observables)
    evals_mitigated, evals_noisy, error_sums = CLP_ZNE_mitigate(circ, observables, layouts, backend, noise_model)
    all_evals_ideal.append(evals_ideal)
    all_evals_mitigated.append(evals_mitigated)
    all_evals_noisy.append(evals_noisy)
    all_error_sums.append(error_sums)